In [5]:
import json
import sys

sys.path.insert(0, '../')
from src.mlflow_utils import mlflow, get_run_list, download_run_data
from src.utils import sanitize_prediction, ResponseDataset
from tqdm.notebook import tqdm

from typing import List, Dict, Tuple

In [6]:
test = ResponseDataset(experiments=[0, 1])

  0%|          | 0/21 [00:00<?, ?it/s]

In [4]:
test = ResponseDataset(runs=["14ae3307f7df4c26b249b847ef24c26a", "71254e7e5ced4ce2a27c5de31f7fb7ab", "6bf2254afdc2439ebd8f566ab84b5b82", "986ee36c268d424ebc061d646c5eadc7"])
with open("label_studio_2long2short.json", 'w+') as f:
    json.dump(test.export_label_studio(), f)

  0%|          | 0/4 [00:00<?, ?it/s]

In [155]:
with open("label_studio_data.json", 'w+') as f:
    json.dump(test.export_label_studio(), f)

In [142]:
with open("nudossi_example.json", 'w+') as f:
    json.dump(test.search_title("Nudossi")[0], f)

In [2]:
selected_experiments = [0, 1]

In [3]:
runs = get_run_list(selected_experiments)

In [47]:
overview = {}

for run in tqdm(runs):
    preds = download_run_data(run)
    for entry in preds['data']:
        x = GT(*entry[1:3])
        if x not in overview:
            overview[x] = {}
        overview[x][run] = sanitize_prediction(entry[3])

  0%|          | 0/21 [00:00<?, ?it/s]

In [7]:
test.search_title("Nudossi")

[({'title': 'Nudossi',
   'context_word': 'Nudossi',
   'context_sentence': '„Kann ich bitte die (das, den) Nudossi haben?"',
   'ground_truth': 'eine herstellerspezifische Bezeichnung für dessen Variante von Nuss-Nugat-Creme'},
  {'a71e5cfb0a954f63b17c3c7b3c626e64': 'nicht-existierendes Wort (trinkt man wahrscheinlich Kakao)',
   '71254e7e5ced4ce2a27c5de31f7fb7ab': 'Nudossi ist eine Marke von Milchmädcheneis.\n\n(Anmerkung: Nudossi ist ein historisches deutsches Eisprodukt, das nicht mehr hergestellt wird. Im Beispielsatz ist es eine Marke von Milchmädcheneis.)',
   'e1d3f1f8aace43caa693e9836834504e': 'Nudossi ist eine Marke von Kaugummi in Deutschland.\n\n(Anmerkung: Nudossi ist ein historischer Bezug, da es in den 1980er-Jahren in der DDR sehr beliebt war. Heutzutage ist die Marke nicht mehr weit verbreitet.)',
   'cb8135f768dd4a8285257deb0bc3d912': 'Nudossi ist eine Marke von Schokocreme aus der DDR.\nDie Antwort wäre also: "Nudossi ist eine Marke von Schokocreme" oder einfach "DDR

In [52]:
for i in overview.keys():
    if "Nudossi" != i.title:
        continue
    print(i)
    print(list(overview[i].values()))
    print()

Nudossi [Nudossi] --- „Kann ich bitte die (das, den) Nudossi haben?" ===> eine herstellerspezifische Bezeichnung für dessen Variante von Nuss-Nugat-Creme
['nicht-existierendes Wort (trinkt man wahrscheinlich Kakao)', 'Nudossi ist eine Marke von Milchmädcheneis.\n\n(Anmerkung: Nudossi ist ein historisches deutsches Eisprodukt, das nicht mehr hergestellt wird. Im Beispielsatz ist es eine Marke von Milchmädcheneis.)', 'Nudossi ist eine Marke von Kaugummi in Deutschland.\n\n(Anmerkung: Nudossi ist ein historischer Bezug, da es in den 1980er-Jahren in der DDR sehr beliebt war. Heutzutage ist die Marke nicht mehr weit verbreitet.)', 'Nudossi ist eine Marke von Schokocreme aus der DDR.\nDie Antwort wäre also: "Nudossi ist eine Marke von Schokocreme" oder einfach "DDR-Schokocreme".', 'fiktive, magische Substanz, ohne Bedeutung außerhalb des Beispiels', 'Es gibt keine bekannte Verwendung des Begriffs „Nudossi“. Möglicherweise ist dies ein Tippfehler oder ein nicht allgemein bekanntes Wort.', 'I